In [1]:
import pandas as pd
import itertools 
from bs4 import BeautifulSoup
import requests
from requests import get
import time
from random import seed
from random import random
from random import randint

In [ ]:
# Function to find max number of page
def maxPageFinder(siteLink):
    numbers = []
    response = get(siteLink)
    soup = BeautifulSoup(response.text, 'html.parser')
    for s in soup.find_all('a',{"class":"pagination__link"}):
        if s.text.isnumeric() == True:
            pageNumber = int(s.text)
            numbers.append(pageNumber)
    return max(numbers)   
    
# Scraping the html content of listing banners
banners = []
site = 'https://www.pararius.com/apartments/rotterdam'
page = 0
while page <= maxPageFinder(site):
    url = site + '/page-' + str(page)
    response = get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    bannerInfo = soup.find_all('li', class_="search-list__item search-list__item--listing")
    if bannerInfo != []:
        banners.extend(bannerInfo)
    time.sleep(0.5)
    page += 1

# Building dataframe
prices = []
agencies = []
postcodes = []
rooms = []
surfaces = []
labels = []
interiors = []
buildPeriods = []

for block in banners:

    label = block.find_all('a', href = True)[1].text    
    labels.append(label)
    labelDF = (
        pd.DataFrame({'label':labels})
        ['label'].str.split(n = 1, expand = True)
        .rename(columns={0: "type", 1: "address"})
    )
    labelDF['address'] = labelDF['address'].str.split("\n").str[0]

    price = block.find_all('div',{"class":"listing-search-item__price"})[0].text
    prices.append(price)
    priceDF = pd.DataFrame({'price':prices})
    priceDF['price'] = priceDF['price'].replace(r'\D+', "",regex=True)
    
    
    agency = block.find_all('a', href=True)[2].text  
    agencies.append(agency)
    agencyDF = pd.DataFrame({'agency':agencies})
    
    postcode = block.find('div',{"class":"listing-search-item__sub-title"}).text
    postcodes.append(postcode)
    postcodeDF = pd.DataFrame({'postcode':postcodes})
    postcodeDF['postcode'] = (postcodeDF['postcode'].str.split('\n')
        .str[1].astype('str')
        .replace('            ',"", regex = True).str[0:7]
    )
    
    room = block.find_all('li',{"class":"illustrated-features__item illustrated-features__item--number-of-rooms"})[0].text
    rooms.append(room)
    roomDF = pd.DataFrame({'rooms':rooms})
    roomDF['rooms'] = roomDF['rooms'].replace(r'\D+', "",regex=True)
    
    surface = block.find_all('li',{"class":"illustrated-features__item illustrated-features__item--surface-area"})[0].text
    surfaces.append(surface)
    surfaceDF = pd.DataFrame({'surface':surfaces})
    surfaceDF['surface'] = surfaceDF['surface'].replace(r'\D+', "",regex=True)

    if block.find_all('li',{"class":"illustrated-features__item illustrated-features__item--interior"}) != []:
        interior = block.find_all('li',{"class":"illustrated-features__item illustrated-features__item--interior"})[0].text
        interiors.append(interior)
    else:
        interiors.append("Unspecified")
    interiorDF = pd.DataFrame({'interior':interiors})

df = pd.concat([labelDF, postcodeDF, surfaceDF, roomDF, agencyDF, interiorDF, priceDF],axis = 1)

In [88]:
df.to_csv('housing.csv')